In [1]:
import pandas as pd
import numpy as np
import glob
import functions
from tqdm import tqdm

In [2]:
# Load jobad ID's along with firm names and correct these using the firm_name() function from functions module
list_of_dfs = []

for file in tqdm(glob.glob(r'H:\HBS Data\Drift III\data\version1.0\cvr\output/*')):
        temp = pd.read_csv(file
                           , sep='\t'
                           , encoding='utf-8'
                           , usecols = ['ID', 'firma', 'dato']
                          )
        df = functions.firm_name(temp)
        list_of_dfs.append(df)
        
data = pd.concat(list_of_dfs)

100%|████████████████████████████████████████████████████████████████████████████████| 170/170 [04:45<00:00,  1.68s/it]


In [3]:
# Load BERT categories for each jobad ID
list_of_dfs = []

for file in tqdm(glob.glob(r'H:\HBS Data\Drift III\data\version1.0\kompetence\kompetenceKategori/*')):
    if 'Genvej' in file:
        pass
    else:
        temp = pd.read_csv(file
                           , sep='\t'
                           , encoding='utf-8'
                           , usecols = ['ID', 'Komp',  'KompNiveau1']
                          )
        list_of_dfs.append(temp)

categories = pd.concat(list_of_dfs)

100%|████████████████████████████████████████████████████████████████████████████████| 169/169 [03:05<00:00,  1.10s/it]


In [4]:
# Combine data
firm_and_skills = pd.merge(data, categories, on='ID', how='left')
firm_and_skills.reset_index(inplace=True, drop=True)

In [5]:
firm_and_skills

,ID,firma,dato,real_firm_name,Komp,KompNiveau1
0,k20070117h1174932607,Zealand Pharma A/S,2007-01-17,Zealand Pharma,test af peptidbaserede lægemidler,"Indsamle, opbevare, overvåge og anvende inform..."
1,k20070117h1174932607,Zealand Pharma A/S,2007-01-17,Zealand Pharma,asning af dyr og,Håndtere og transportere
2,k20070117h1174932607,Zealand Pharma A/S,2007-01-17,Zealand Pharma,opstaldningsfaciliteterassistance ved in vivo ...,Pleje og omsorg for andre
3,k20070117h1174932607,Zealand Pharma A/S,2007-01-17,Zealand Pharma,uddannet dyrepassererfaring gnavere og doserin...,Håndtere og transportere
4,k20070117h1174932607,Zealand Pharma A/S,2007-01-17,Zealand Pharma,assistance ved dyreforsøgvant at arbejde med e...,Arbejde med computere
...,...,...,...,...,...,...
2169868,k2021020893770354943,ALK-Abelló A/S,2021-02-08,ALK-Abelló B,NaN,NaN
2169869,k2021021093818724943,ALK-Abelló A/S,2021-02-10,ALK-Abelló B,NaN,NaN
2169870,k2021021893976974386,ALK-Abelló A/S,2021-02-18,ALK-Abelló B,NaN,NaN
2169871,k2021021893976993898,ALK-Abelló A/S,2021-02-18,ALK-Abelló B,NaN,NaN


In [6]:
# Dataframe with most frequent skill in jobad
most_frequent = pd.DataFrame(firm_and_skills.groupby('ID').KompNiveau1.value_counts())\
                            .rename(columns={'KompNiveau1':'antal'})\
                            .reset_index()

most_frequent = most_frequent.drop_duplicates('ID', keep='first')
most_frequent.head(3)

,ID,KompNiveau1,antal
0,k2007010215674142749,"Indsamle, opbevare, overvåge og anvende inform...",4
4,k2007010215674152687,Arbejde med computere,4
8,k2007010215675372693,Værdier og holdninger,10


In [7]:
firm_name = firm_and_skills[['ID', 'real_firm_name', 'dato']].drop_duplicates('ID')
final_firm_data = pd.merge(most_frequent, firm_name, on='ID', how='left')
final_firm_data.head(3)

,ID,KompNiveau1,antal,real_firm_name,dato
0,k2007010215674142749,"Indsamle, opbevare, overvåge og anvende inform...",4,Lollands Bank,2007-01-02
1,k2007010215674152687,Arbejde med computere,4,Lollands Bank,2007-01-02
2,k2007010215675372693,Værdier og holdninger,10,SAS,2007-01-02


In [8]:
# Read stock data
stocks = pd.read_csv(r'pris_data.csv', usecols=['dato', 'adj_close', 'firma'])
stocks = stocks.rename(columns={'firma':'real_firm_name'})
stocks = stocks[stocks.adj_close.notna()]
stocks.head(3)

,dato,real_firm_name,adj_close
4175,2016-06-09,Ørsted,234.955612
4176,2016-06-10,Ørsted,229.491516
4177,2016-06-13,Ørsted,222.206085


In [9]:
event_test = pd.merge(stocks, final_firm_data, on=['real_firm_name', 'dato'], how='left')
event_test.head(3)

,dato,real_firm_name,adj_close,ID,KompNiveau1,antal
0,2016-06-09,Ørsted,234.955612,NaN,NaN,NaN
1,2016-06-10,Ørsted,229.491516,NaN,NaN,NaN
2,2016-06-13,Ørsted,222.206085,NaN,NaN,NaN


In [10]:
jodad_dummy = []

for i in event_test.KompNiveau1:
    if type(i)==str:
        jodad_dummy.append(1)
    else:
        jodad_dummy.append(0)

event_test['jodad_dummy'] = jodad_dummy
event_test.head(3)

,dato,real_firm_name,adj_close,ID,KompNiveau1,antal,jodad_dummy
0,2016-06-09,Ørsted,234.955612,NaN,NaN,NaN,0
1,2016-06-10,Ørsted,229.491516,NaN,NaN,NaN,0
2,2016-06-13,Ørsted,222.206085,NaN,NaN,NaN,0


In [11]:
mapping = {}
list_of_cat_names = ['communication', 'values', 'information', 'computer', 'leadership','machines', 'leadership', 'handling_and_transport', 'caring','build_and_construct']
for name,cat in zip(list_of_cat_names, event_test.KompNiveau1.value_counts().index.tolist()):
    mapping[cat] = name
event_test['category'] = event_test.KompNiveau1.map(mapping)
event_test['category_dummy'] = event_test.KompNiveau1.map(mapping)

In [12]:
event_test = pd.get_dummies(event_test, columns=['category_dummy'])
event_test.to_csv('event_test_data.csv')

In [111]:
mapping

{'Kommunikere, samarbejde og udøve kreativitet': 0,
 'Værdier og holdninger': 1,
 'Indsamle, opbevare, overvåge og anvende information': 2,
 'Arbejde med computere': 3,
 'Lede mennesker aktiviteter ressourcer og organisationer': 4,
 'Betjene maskiner, køretøjer og specialudstyr': 5,
 'Lede mennesker, aktiviteter, ressourcer og organisationer': 6,
 'Håndtere og transportere': 7,
 'Pleje og omsorg for andre': 8,
 'Bygge og konstruere': 9}

In [81]:
pd.get_dummies(event_test, columns=['encoded_cat'])

,dato,real_firm_name,adj_close,ID,KompNiveau1,antal,jodad_dummy,encoded_cat_-1,encoded_cat_0,encoded_cat_1,encoded_cat_2,encoded_cat_3,encoded_cat_4,encoded_cat_5,encoded_cat_6,encoded_cat_7,encoded_cat_8,encoded_cat_9
0,2016-06-09,Ørsted,234.955612,NaN,NaN,NaN,0,1,0,0,0,0,0,0,0,0,0,0
1,2016-06-10,Ørsted,229.491516,NaN,NaN,NaN,0,1,0,0,0,0,0,0,0,0,0,0
2,2016-06-13,Ørsted,222.206085,NaN,NaN,NaN,0,1,0,0,0,0,0,0,0,0,0,0
3,2016-06-14,Ørsted,227.123749,NaN,NaN,NaN,0,1,0,0,0,0,0,0,0,0,0,0
4,2016-06-15,Ørsted,230.857529,NaN,NaN,NaN,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615865,2021-03-08,A.P. Møller - Mærsk A,12350.000000,NaN,NaN,NaN,0,1,0,0,0,0,0,0,0,0,0,0
615866,2021-03-09,A.P. Møller - Mærsk A,12990.000000,NaN,NaN,NaN,0,1,0,0,0,0,0,0,0,0,0,0
615867,2021-03-10,A.P. Møller - Mærsk A,13090.000000,NaN,NaN,NaN,0,1,0,0,0,0,0,0,0,0,0,0
615868,2021-03-11,A.P. Møller - Mærsk A,13290.000000,NaN,NaN,NaN,0,1,0,0,0,0,0,0,0,0,0,0


In [7]:
firm_and_skills[firm_and_skills.ID=='k20070117h1174932607']

,ID,firma,dato,real_firm_name,Komp,KompNiveau1
0,k20070117h1174932607,Zealand Pharma A/S,2007-01-17,Zealand Pharma,test af peptidbaserede lægemidler,"Indsamle, opbevare, overvåge og anvende inform..."
1,k20070117h1174932607,Zealand Pharma A/S,2007-01-17,Zealand Pharma,asning af dyr og,Håndtere og transportere
2,k20070117h1174932607,Zealand Pharma A/S,2007-01-17,Zealand Pharma,opstaldningsfaciliteterassistance ved in vivo ...,Pleje og omsorg for andre
3,k20070117h1174932607,Zealand Pharma A/S,2007-01-17,Zealand Pharma,uddannet dyrepassererfaring gnavere og doserin...,Håndtere og transportere
4,k20070117h1174932607,Zealand Pharma A/S,2007-01-17,Zealand Pharma,assistance ved dyreforsøgvant at arbejde med e...,Arbejde med computere
5,k20070117h1174932607,Zealand Pharma A/S,2007-01-17,Zealand Pharma,nsvarsbevidst,Værdier og holdninger
6,k20070117h1174932607,Zealand Pharma A/S,2007-01-17,Zealand Pharma,fleksibel rette pioneråndgode,Værdier og holdninger
7,k20070117h1174932607,Zealand Pharma A/S,2007-01-17,Zealand Pharma,samarbejdsevnerudadvendt,"Kommunikere, samarbejde og udøve kreativitet"
8,k20070117h1174932607,Zealand Pharma A/S,2007-01-17,Zealand Pharma,engagement,Værdier og holdninger
9,k20070117h1174932607,Zealand Pharma A/S,2007-01-17,Zealand Pharma,sans for humorvi,Værdier og holdninger


In [ ]:
df.real_firm_name.value_counts()

In [ ]:
df[df.real_firm_name != 'delete']

In [ ]:
for name,list_ in zip(list_of_names,list_of_lists):
    print(name, '          ', list_[0])